# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("../output_data/cities.csv")
del df["Unnamed: 0"]
df

,CITY,LAT,LNG,TEMP,MAXTEMP,HUMIDITY,CLOUDINESS,WINDSPEED
0,Richards Bay,-28.7830,32.0377,70.39,70.39,61,0,14.92
1,Punta Arenas,-53.1500,-70.9167,42.03,42.03,72,0,39.12
2,Hilo,19.7297,-155.0900,72.25,72.25,70,0,6.91
3,Salalah,17.0151,54.0924,77.09,77.09,88,75,5.75
4,Kahului,20.8947,-156.4700,71.62,71.62,68,20,14.00
5,Arroio Grande,-32.2375,-53.0869,56.19,56.19,64,83,13.15
6,Bredasdorp,-34.5322,20.0403,50.85,50.85,72,100,11.27
7,Mataura,-46.1927,168.8643,32.56,32.56,97,67,2.30
8,Albany,42.6001,-73.9662,84.33,84.33,40,52,1.99
9,Kutum,14.2000,24.6667,79.38,79.38,55,100,8.08


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key) 

In [4]:
locations = df[["LAT", "LNG"]]

weights = df["HUMIDITY"].astype(float)

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [6]:
best_cities_df = df.loc[ (df["TEMP"].between(69, 84)) & (df["HUMIDITY"] < 80) & (df["CLOUDINESS"] < 50), :]
best_cities_df.dropna()
best_cities_df

,CITY,LAT,LNG,TEMP,MAXTEMP,HUMIDITY,CLOUDINESS,WINDSPEED
0,Richards Bay,-28.7830,32.0377,70.39,70.39,61,0,14.92
2,Hilo,19.7297,-155.0900,72.25,72.25,70,0,6.91
4,Kahului,20.8947,-156.4700,71.62,71.62,68,20,14.00
16,Atuona,-9.8000,-139.0333,76.80,76.80,75,19,23.06
20,Rikitea,-23.1203,-134.9692,72.55,72.55,74,5,18.52
23,Bouaké,7.6895,-5.0218,80.71,80.71,65,20,6.91
31,Arraial do Cabo,-22.9661,-42.0278,78.76,78.76,69,20,18.41
40,Saint-Michel-des-Saints,46.6834,-73.9159,69.94,69.94,56,49,11.07
41,Mahébourg,-20.4081,57.7000,70.09,70.09,73,20,9.22
46,Gigante,2.3868,-75.5453,82.63,82.63,50,19,2.26


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
hotel_df = df.copy()

hotel_df["HOTEL_NAME"] = "NULL"
hotel_df["HOTEL_CITY"] = "NULL"
hotel_df["HOTEL_COUNTRY"] = "NULL"

hotel_df = hotel_df.head(10)

In [22]:
# params dictionary to update each iteration

params = {
    "location": "0,0",
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports

for index, row in hotel_df.iterrows():
    
    # get lat, lng from df
    
    lat = row["LAT"]
    lng = row["LNG"]

    # change location each iteration while leaving original params in place
    
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_data = requests.get(base_url, params=params)

    # convert to json
    hotel_json = hotel_data.json()
    pprint(hotel_json)

#     # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.

    try:
        hotel_df.loc[index, "HOTEL_NAME"] = hotel_json["results"][0]["name"]
        hotel_df.loc[index, "HOTEL_CITY"] = hotel_json["results"][0]["plus_code"]["compound_code"].split(",")[0].split(" ", 1)[1]
        hotel_df.loc[index, "HOTEL_COUNTRY"] = hotel_json["results"][0]["plus_code"]["compound_code"].split(",")[1]
        # print(f'HOTEL TO ADD: {hotel_json["results"][0]["name"]}')
        
    except (KeyError, IndexError):
        
        print("Missing field/result... skipping.")

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -28.7933721, 'lng': 32.080771},
                           'viewport': {'northeast': {'lat': -28.79192512010727,
                                                      'lng': 32.08232197989272},
                                        'southwest': {'lat': -28.79462477989272,
                                                      'lng': 32.07962232010727}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': 'BON Hotel Waterfront Richards Bay',
              'opening_hours': {'open_now': True},
              'photos': [{'height': 4912,
                          'html_attributions': ['<a '
                                                'href="https

{'html_attributions': [],
 'next_page_token': 'AeJbb3eWHXE0e7-y5XIPM8LjNvOA-64Hv7ROAHBw4YowjLE374XSzJLTH31wog5qX22juTt5w4vW8mFbDAaKIcKC6qqiT3WCod6r_4t40T8qToRz46kDxrOwqfCN9c2OhIFdKawWQGa9pJ0g-KFojiSP_dxPfOUZobBbyMEuyxdy592SwDmp_YCsXWcDx8nsPJImmscdo8y2_GkWTrqRNf3N1rndFS1v_AH5poFN5-fCJl5Ebna-xC_ZUsoqQTNPeT6vLnJXTma-1j9JYX06ZZYFQdQeezZM9SLHH_XBjWcvHszKnHHiKXbDuq9OyaLEGhZZgi6baW7JQL8oEo3QB4NXZvcL276rEHl0y0bHnpqW10V8CBQyIcQRo5kocdyMB1PzTOufgGTACrPsw_QzkT5RxoKdvlAx7M0PgKhTbV0bvDrlAD8JRN2B8eUWKa2E3Wf-4w',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -53.1628664, 'lng': -70.9069727},
                           'viewport': {'northeast': {'lat': -53.16150127010727,
                                                      'lng': -70.90565937010729},
                                        'southwest': {'lat': -53.16420092989271,
                                                      'lng': -70.90835902989272}}},
              'icon': 'https://maps.gst

{'html_attributions': [],
 'next_page_token': 'AeJbb3ewpvbxqwBeD1DpQNvUCxYfeCyM7mMr9LJ5-1v4Yj1UYGnog40yzq3KisH-nZox4hgCiQEN7zDP_sQoZf3MqW5EDfk4IxxcRJg74ra-6EjxLbSznIDibmhEG-LveAQgC4_wjMKGvlE4HYKC7MlrtzZbDTqQdKe_y54BV54K9LDtlM1C4RDENC48cZHoBmOypbnGU6xjd0peCZHWWhuQTyZmut0Op_zTTKuob_C9ZycwPO2rNMsOU6FfZapyTH2y9UQAPOSJn03IngyGWTqSSAd46LqCNKzKLSVpoHr8u-FxpmZbauG98ew4Dp1E0oH4pJSHxqeKLKX_WS2FEs6uIZBflppFJTU8W8kM5eEk-l17cLoTCaTFewfrpPDc3m7RvCr4WWkqy4nbJO6J3PU9YVEYCYRcUstBAuixtuOy9y4VWHcAoAToQ6eImYGCPj2PXw',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 19.7283446, 'lng': -155.0664264},
                           'viewport': {'northeast': {'lat': 19.72944692989272,
                                                      'lng': -155.0650755701072},
                                        'southwest': {'lat': 19.72674727010728,
                                                      'lng': -155.0677752298927}}},
              'icon': 'https://maps.gstat

              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': 'Minshuku Hilo',
              'opening_hours': {'open_now': True},
              'photos': [{'height': 2213,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/115233127809507179074">Minshuku '
                                                'Hilo (民宿HILO)</a>'],
                          'photo_reference': 'AeJbb3dBiH5PdjuvvIAETUrdqf7lk9NZQi58R3arE7MKwK2J-lRfsy6mplz8noCb9gqvn_YwTFi9Ci5IcYRTpW6-666DAgybcujYUEw72jXOXVzhgywfpUXuqoAXMB-iQj5BGpAptW3qX1QCNLqk9-h7-TdF4Z8zxhlPHFRL8IwaXGXwJFyh',
                          'width': 3319}],
              'place_id': 'ChIJqaTtkmhLUnkRS1NlC3ZaEOU',
              'plus_code': {'compound_code': 'PW6J+3M Hilo, Hawaii',
                            'global_code': '73F6PW6J+3M'},
              'rating': 4.4,
              'reference': 

              'rating': 3.4,
              'reference': 'ChIJWQBzkgFg0T0RfUbsZkMAT4s',
              'scope': 'GOOGLE',
              'types': ['lodging', 'point_of_interest', 'establishment'],
              'user_ratings_total': 25,
              'vicinity': '23CM+G7V, Salalah'},
             {'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 17.0163454,
                                        'lng': 54.10937449999999},
                           'viewport': {'northeast': {'lat': 17.01765307989272,
                                                      'lng': 54.11072837989272},
                                        'southwest': {'lat': 17.01495342010728,
                                                      'lng': 54.10802872010727}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -32.2426913, 'lng': -53.0702468},
                           'viewport': {'northeast': {'lat': -32.24129822010728,
                                                      'lng': -53.06885792010727},
                                        'southwest': {'lat': -32.24399787989272,
                                                      'lng': -53.07155757989271}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': 'Hotel Amsterdam',
              'photos': [{'height': 3096,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/109681119546982902799">Leandro '


                          'photo_reference': 'AeJbb3d23L5g6GWn0-U87LC-uOlrkNgUHqVW-gvwoOdRE7Ed9Y4WSjudzuAhLAgL9nLbK_3-8vboxe-We2UJcH2T_Cfsh_CMPwfNIBDhV6kPdrTdCP-sR-gRglZMUbK_oD2rKM8AodGpz4ehTBiurGM2K6fwDGcav9B2Vj9Ey8Dh7kKPsEpE',
                          'width': 4608}],
              'place_id': 'ChIJhWyBXyGX0R0RSlKl7IwIh0U',
              'plus_code': {'compound_code': 'G37V+QF Bredasdorp, South Africa',
                            'global_code': '4GQ2G37V+QF'},
              'rating': 5,
              'reference': 'ChIJhWyBXyGX0R0RSlKl7IwIh0U',
              'scope': 'GOOGLE',
              'types': ['lodging', 'point_of_interest', 'establishment'],
              'user_ratings_total': 2,
              'vicinity': 'Bredasdorp'},
             {'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -34.5346401, 'lng': 20.0397874},
                           'viewport': {'northeast': {'lat': -34.53326407010728,
                                                  

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 58.5189507, 'lng': -117.1203477},
                           'viewport': {'northeast': {'lat': 58.52045527989272,
                                                      'lng': -117.1189940201073},
                                        'southwest': {'lat': 58.51775562010728,
                                                      'lng': -117.1216936798927}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': 'Best Western Plus Mirage Hotel & Resort',
              'photos': [{'height': 3024,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/10970023454

{'html_attributions': [],
 'next_page_token': 'AeJbb3cs5cH9YNdtmJlm81gedBhhjXJU6woI7fBd1KiBbwx2hXWNMT3ZJvw1ny64Hi3qZB29-thgg6ydIcO7tX1gkDQHTR4sZN7uwksPHL3d7yE7pKO1MvCGdzsc2Ob6zLulVXRL72I4D-Md5M54YwcOdGGNAmkveBKClTnHbUHJkGje-NnyaKiPqcanc95IFq-DFQiWt7M7iFr1GO3FjUKZl5_ZPYpVppNN6J63ThRK_hwVC_Aqhlmc56NtR6ekqhTjZz0QQHm1pN38VLxjhl1lk2r2nBMzp-adB6cE4BKycddvahPAgiywH_fN7cpZ4edOHKo9HgVvsCiC_i2rW8oLb_oIXhc5_P_z6fZckqjEnu4sxCSwuGq6faS4zrRnOMN4wf4R0jpbQPHHTncFDQ4Y7XdnRSAe3Pf7fNBiMIBGirjO0WC63bWdG6gHGEOkWbPZRg',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -54.8074811, 'lng': -68.3086124},
                           'viewport': {'northeast': {'lat': -54.80608367010728,
                                                      'lng': -68.30729862010728},
                                        'southwest': {'lat': -54.80878332989273,
                                                      'lng': -68.30999827989272}}},
              'icon': 'https://maps.gst

              'reference': 'ChIJiY5NgxgjTLwRbbAUnwTn-r0',
              'scope': 'GOOGLE',
              'types': ['lodging', 'point_of_interest', 'establishment'],
              'user_ratings_total': 687,
              'vicinity': 'Av. San Martín 1299, Ushuaia'},
             {'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -54.8043975, 'lng': -68.3571472},
                           'viewport': {'northeast': {'lat': -54.80319797010728,
                                                      'lng': -68.35546852010727},
                                        'southwest': {'lat': -54.80589762989272,
                                                      'lng': -68.35816817989271}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': '

In [24]:
df = hotel_df.loc[   hotel_df["HOTEL_NAME"] != "NULL", :]
df.reset_index(drop=True)
df


,CITY,LAT,LNG,TEMP,MAXTEMP,HUMIDITY,CLOUDINESS,WINDSPEED,HOTEL_NAME,HOTEL_CITY,HOTEL_COUNTRY
0,Richards Bay,-28.7830,32.0377,70.39,70.39,61,0,14.92,BON Hotel Waterfront Richards Bay,Richards Bay,South Africa
1,Punta Arenas,-53.1500,-70.9167,42.03,42.03,72,0,39.12,Hotel Cabo de Hornos,Punta Arenas,Chile
2,Hilo,19.7297,-155.0900,72.25,72.25,70,0,6.91,Hilo Hawaiian Hotel,Hilo,Hawaii
3,Salalah,17.0151,54.0924,77.09,77.09,88,75,5.75,Salalah Gardens Hotel,Salalah,Oman
4,Kahului,20.8947,-156.4700,71.62,71.62,68,20,14.00,Maui Beach Hotel,Kahului,Hawaii
5,Arroio Grande,-32.2375,-53.0869,56.19,56.19,64,83,13.15,Hotel Amsterdam,Arroio Grande,RS
6,Bredasdorp,-34.5322,20.0403,50.85,50.85,72,100,11.27,Victoria Hotel - Bredasdorp,Bredasdorp,South Africa
7,Mataura,-46.1927,168.8643,32.56,32.56,97,67,2.30,Ellie's Villa,Mataura,New Zealand
10,High Level,58.5169,-117.1360,66.31,66.31,77,75,8.05,Best Western Plus Mirage Hotel & Resort,High Level,Alberta
11,Ushuaia,-54.8000,-68.3000,42.46,42.46,56,20,10.36,Hotel Lennox,Ushuaia,Tierra del Fuego Province


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{HOTEL_NAME}</dd>
<dt>City</dt><dd>{HOTEL_CITY}</dd>
<dt>Country</dt><dd>{HOTEL_COUNTRY}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

hotel_info = [info_box_template.format(**row) for index, row in df.iterrows()]
locations = df[["LAT", "LNG"]]

In [27]:
# Add marker layer ontop of heat map

 # Assign the marker layer to a variable
    
markers = gmaps.marker_layer(locations=locations, display_info_box=True, info_box_content=hotel_info)

# Add the layer to the map

fig.add_layer(markers)
fig


# Display figure


Figure(layout=FigureLayout(height='420px'))